In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from aana.sdk import AanaSDK


aana_app = AanaSDK()

/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-08 15:12:38,762	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [3]:
aana_app.connect()

In [5]:
from aana.deployments.hf_pipeline_deployment import (
    HfPipelineDeployment,
    HfPipelineConfig,
)


hf_pipeline_deployment = HfPipelineDeployment.options(
    num_replicas=1,
    ray_actor_options={"num_gpus": 1},
    user_config=HfPipelineConfig(model_id="ydshieh/vit-gpt2-coco-en").model_dump(
        mode="json"
    ),
)

In [6]:
aana_app.register_deployment("hf_pipeline", hf_pipeline_deployment, deploy=True)

In [8]:
from aana.deployments.aana_deployment_handle import AanaDeploymentHandle


handle = await AanaDeploymentHandle.create("hf_pipeline")

In [22]:
from PIL import Image as PilImage
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"

image = PilImage.open(requests.get(url, stream=True).raw)

In [11]:
await handle.call(images=[image], generate_kwargs={"max_new_tokens": 200})

[[{'generated_text': 'a cat laying on a blanket next to a cat laying on a bed '}]]

In [13]:
await handle.call(image, generate_kwargs={"max_new_tokens": 200})

[{'generated_text': 'a cat laying on a blanket next to a cat laying on a bed '}]

In [14]:
await handle.call(url, generate_kwargs={"max_new_tokens": 200})

[{'generated_text': 'a cat laying on a blanket next to a cat laying on a bed '}]